In [1]:
!pip install kumaraswamy

In [2]:
from kumaraswamy import kumaraswamy

## Experiments with Kumaraswamy Function

In [3]:
d1 = kumaraswamy(a=0.1, b=0.7)

In [4]:
d1.pdf(0.9)

array(0.30210796)

In [5]:
d1.cdf(0.21)

array(0.7418337)

In [6]:
d1.var

0.07907948744725794

In [7]:
d1.mean

0.17121255852442666

### CCTE

In [8]:
from scipy.special import hyp2f1
from scipy.special import betainc
#The incomplte Beta function


In [9]:
def num_1(alpha,t,a,b,theta):
    v1=-(1/a)
    v2=b
    v3=1+b
    v4=(1-alpha)**(1/b)
##This is the first term in the numerator
    p1=(t-1)*(theta*t-1)
    p2=(1-alpha)*(hyp2f1(v1,v2,v3,v4))
    return p1*p2 #The product of 2  terms is the first term
    

In [10]:
def num_2(alpha,t,a,b,theta):
    #This gives the negative of the second term
    a3=1+(1/a)
    p=alpha**b
    b3=2*b
    p3=2*(theta)*t*(t-1)*b
    p4=[betainc(a3,b,1)+betainc(a3,b3,1)-betainc(a3,b,p)-betainc(a3,b3,p)]
    prod=p4[0]*p3
    return prod


In [11]:
#This defines the FGM Copula for kumaraswamy distribution
# Some checks required
def FGM_copula(alpha,t,a,b,theta,a1,b1):
    d1 = kumaraswamy(a, b)
    d2 = kumaraswamy(a1, b1)
    s1 = d1.cdf(alpha)
    s2 = d2.cdf(t)
    copula=(s1)*(s2)+theta*(s1)*(s2)*(1-s1)*(1-s2)
    return copula
    
    
    
def deno(alpha,t,a,b,theta,a1,b1):
    t1=1-alpha-t
    t2= FGM_copula(alpha,t,a,b,theta,a1,b1)
    return t1+t2
    

In [12]:
FGM_copula(0.3,0.5,0.7,0.8,2,0.6,0.8)

0.32232193347566945

In [13]:
#Here a,b, alpha are for first distribution and a1,b1,t are for second distribution
def ccte_fun(alpha,a,b,t,a1,b1,theta):
    num=num_1(alpha,t,a,b,theta)+num_2(alpha,t,a,b,theta)
    den=deno(alpha,t,a,b,theta,a1,b1)
    return num/den
    

In [14]:
ccte_fun(0.2,0.5,0.8,0.9,0.8,0.9,0.4)

-0.3503350944639705

In [15]:

num_2(0.2,0.7,0.8,0.9,4)

-2.838277127208187

In [16]:
import numpy as np
def matrix(n):
    a=(n,n)
    return np.zeros(a)

In [17]:
c=matrix(3)

In [18]:
c

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [19]:
c[1,2]=25

In [20]:
c

array([[ 0.,  0.,  0.],
       [ 0.,  0., 25.],
       [ 0.,  0.,  0.]])

In [21]:
def update_matrix(mat,i,j,alpha,a,b,t,a1,b1,theta):
    #input here whcih element we want to update(exact index)
    mat[i-1,j-1]=ccte_fun(alpha,a,b,t,a1,b1,theta)
    

In [22]:
update_matrix(c,2,2,0.2,0.5,0.8,0.9,0.8,0.9,0.4)

In [23]:
c

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.        , -0.35033509, 25.        ],
       [ 0.        ,  0.        ,  0.        ]])

### The OOP part

In [24]:
class ccte:
    """
        A class used to get ccte of bivariate kumaraswamy distribution
        
        ...
        
        Attributes
        ------
        alpha: float
            The percentile for the first kumaraswamy variable
        a,b: float
            The kumaraswamy distribution parameters for the first variable
        t: float    
            The percentile for the first kumaraswamy variable
        a1,b1: float
            The kumaraswamy distribution parameters for the second variable
        ccte_value : float
            Stores the value of CCTE
        
        
        
        Methods
        ------
        ccte_calc:float
            Stores the ccte of first varible with repect to another 
            This assumes both the variables are kumaraswamy and copula used is FGM copula
            Stores the value of CCTE in ccte_value
    """

    def __init__(self, alpha,a,b,t,a1,b1):
       
        """
            Parameters
            ------
            alpha: float
                The percentile for the first kumaraswamy variable
            a,b: float
                The kumaraswamy distribution parameters for the first variable
            t: float    
                The percentile for the first kumaraswamy variable
            a1,b1: float
                The kumaraswamy distribution parameters for the second variable
        """
        self.alpha = alpha
        self.a = a
        self.b = b
        self.t = t
        self.a1 = a1
        self.b1 = b1
     
             
       
            
        
       
       
        
        
    def ccte_calc(self,theta):
        
        """ Returns the ccte(conditional copula tail expectation) for first variable with repect to second
         and the copula used is FGM copula
         
         
        Parameters
        ------
        Theta: float
            The dependence paramater of FGM copula 
        """
        
        alpha=self.alpha
        a=self.a
        b=self.b
        t=self.t
        a1=self.a1
        b1=self.b1
        self.ccte_value=ccte_fun(alpha,a,b,t,a1,b1,theta)
        return ccte_fun(alpha,a,b,t,a1,b1,theta)

In [25]:
obj=ccte(0.2,0.5,0.8,0.9,0.8,0.3)

In [26]:
obj.ccte_calc(0.2)

-0.21156365084895895

In [133]:
class dispersion_matrix:
    
    """
        A class used to store the dispersion matrix of a network
        
        ...
        
        Attributes
        ------
        n: int
            stores the value of size of matrix
        matrix: array(2D)
            Stores the zeros matrix of size n*n
        p: array(2D)
            Stores the updated matrix
            
            
        Methods
        ------
        matrix_def():
            builds a n*n matrix with all entries as zeros
        update_mat(i,j,alpha,a,b,t,a1,b1,theta)
            updates the (i,j) eleemnt of the dispersion matrix with the CCTE of i-th element with repect to jth 
            element
        num_1(alpha,t,a1,b1,theta):
            Gives the first term of the numerator(to be used internally for getting the ccte)
        num_2(self,alpha,t,a1,b1,theta):
            Gives the second term of the numerator(to be used internally for getting the ccte)
        FGM_copula(alpha,t,a1,b1,theta,a2,b2)
            Gives the FGM copula of 2 kumaraswamy distributions given the percentile alpha(corresponding params
            a1 and b1) and t(corresponding params a2 and b2)
        deno(alpha,t,a1,b1,theta,a2,b2)
            Gives the Denominator for ccte(to be used internally for getting the ccte)
        ccte_fun(self,alpha,a1,b1,t,a1,b1,theta)
            Calculates the ccte of kumaraswamy varible with parameters a1,b1 with respect to the one 
            with parameters a2,b2  
       """
    
    
    def __init__(self,n):
        
        """
        Parameters
        ------
        n: Thse size of dispersion matrix
        """
        self.n=n
        
    def num_1(self,alpha,t,k1,theta):
        """
      The function to gives the first term of the numerator to calculate the ccte 
        Parameters
        ------
        alpha: float
            The percentile for the first kumaraswamy variable
        k1: Object
            The kumaraswamy_scaled object which contains kumaraswamy distribution
        t: float    
            The percentile for the first kumaraswamy variable
            
        Returns
        ------
        The first term in the numerator to calculate the ccte
        """
        v1=-(1/k1.a)
        v2=k1.b
        v3=1+k1.b
        v4=(1-alpha)**(1/k1.b)
    
        p1=(t-1)*(theta*t-1)
        p2=(1-alpha)*(hyp2f1(v1,v2,v3,v4))
        return p1*p2 #The product of 2  terms is the first term
    
    def num_2(self,alpha,t,k2,theta):
        """
      The function to gives the neagtive of the second term of the numerator to calculate the ccte
      
        Parameters
        
        ------
        alpha: float
            The percentile for the first kumaraswamy variable
        k1: Object
            The kumaraswamy_scaled object which contains the first kumaraswamy distribution
        k2: Object
            The kumaraswamy_scaled object which contains the second kumaraswamy distribution
        t: float    
            The percentile for the second kumaraswamy variable
            
            
        Returns
        ------
        The denominator to calculate the ccte
        """
        
   
        a3=1+(1/k2.a)
        p=alpha**k2.b
        b3=2*k2.b
        p3=2*(theta)*t*(t-1)*k2.b
        p4=[betainc(a3,k2.b,1)+betainc(a3,b3,1)-betainc(a3,k2.b,p)-betainc(a3,b3,p)]
        prod=p4[0]*p3
        return prod
    
    def FGM_copula(self,alpha,t,k1,theta,k2):
        """
         
        Gives the FGM copula of 2 kumaraswamy distributions given the percentile alpha(corresponding params
        a1 and b1) and t(corresponding params a2 and b2)
       
        Parameters
        
        ------
        alpha: float
            The percentile for the first kumaraswamy variable
        k1: Object
            The kumaraswamy_scaled object which contains first kumaraswamy distribution
        k2: Object
            The kumaraswamy_scaled object which contains second kumaraswamy distribution    
        t: float    
            The percentile for the second kumaraswamy variable
        Theta: float
            The dependence paramter of the FGM copula
            
        Returns
        ------
        The FGM copula between two input kumaraswamy variables with given percentiles
        
        
        """
        d1 = kumaraswamy(k1.a, k1.b)
        d2 = kumaraswamy(k2.a, k2.b)
        s1 = d1.cdf(alpha)
        s2 = d2.cdf(t)
        copula=(s1)*(s2)+theta*(s1)*(s2)*(1-s1)*(1-s2)
        return copula
    
    def deno(self,alpha,t,k1,theta,k2):
        """
        The function calculates the denominator that is used to calculate the ccte internally
        
        Parameters
        
        ------
        alpha: float
            The percentile for the first kumaraswamy variable
        k1: Object
            The kumaraswamy_scaled object which contains first kumaraswamy distribution
        k2: Object
            The kumaraswamy_scaled object which contains second kumaraswamy distribution 
        t: float    
            The percentile for the second kumaraswamy variable
        Theta: float
            The dependence paramter of the FGM copula
            
            
        Returns
        ------
        The neagtive of the second term in the numerator to calculate the ccte
        """
        t1=1.-alpha-t
        t2= self.FGM_copula(alpha,t,k1,theta,k2)
        return t1+t2
    
    def ccte_fun(self,alpha,k1,t,k2,theta):
        """
        Parameters
        
        ------
        alpha: float
            The percentile for the first kumaraswamy variable
        k1: Object
            The kumaraswamy_scaled object which contains first kumaraswamy distribution
        k2: Object
            The kumaraswamy_scaled object which contains second kumaraswamy distribution 
        t: float    
            The percentile for the first kumaraswamy variable
        Theta: float
            The dependence paramter of the FGM copula
            
        Returns
        
        --------
        
        Calculates the ccte of kumaraswamy varible with parameters a1,b1 with respect to the one 
        with parameters a2,b2 
        """
        num=self.num_1(alpha,t,k1,theta)-self.num_2(alpha,t,k1,theta)
        den=self.deno(alpha,t,k1,theta,k2)
        return num/den
    
        
    def matrix_def(self):
        
        """  builds a n*n matrix with all entries as zeros
         
         
        Parameters
        ------
        None
        
        
        Returns
        -------
        A n*n matrix with all zeroes
        
        """
        self.matrix=(self.n,self.n)
        self.p=np.zeros(self.matrix)
        return self.p
    
    def update_mat(self,i,j,alpha,k1,t,k2,theta):
        """ Updates the (i,j) element of a matrix with the CCTE
         Stores the updated matrix in p(attribute)
        Parameters
        ------
        alpha: float
            The percentile for the first kumaraswamy variable
        a1,b1: float
            The kumaraswamy distribution parameters for the first variable
        t: float    
            The percentile for the first kumaraswamy variable
        a2,b2: float
            The kumaraswamy distribution parameters for the second variable
            
        Returns
        ------
        An updated matrix p
         """
        
        self.p[i-1,j-1]=self.ccte_fun(alpha,k1,t,k2,theta)
        self.p[j-1,i-1]=self.ccte_fun(alpha,k2,t,k2,theta)
        return self.p


In [134]:
obj2=dispersion_matrix(3)

In [135]:
obj2.ccte_fun(0.2,obj3,0.9,obj3,0.4)

0.5498635456349172

In [136]:
obj2.matrix_def()

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [140]:
obj2.update_mat(1,2,0.8,obj3,0.9,obj4,0.4)

array([[ 0.        , -0.09660801,  0.        ],
       [-0.05218522,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ]])

In [141]:
from kumaraswamy import kumaraswamy
class kumaraswamy_scaled:
    """
    The Class that stores scaled kumaraswamy distribution
     ...
        
        Attributes
        ------
        a,b: Float 
            parameters of Kumaraswamy Distribution
        upper, lower: Float
            Stores the upper and lower limits of the disrtibution
        z: Float
            Stores scaled and shifted value of x(the input)
        pdf: Float
            Stores pdf of the distribution at some value x
        cdf: Float
            Stores cdf of the distribution at some value x
        mean: Float
             Stores mean of the distribution
        var: Float
              Stores variance of the distribution
            
        Methods
        ------
        get_z(x):
            gets the z(normalized)(scaled and shifted) value for corrsponding x
        get_pdf(x):
            returns and stores pdf of a value x
        get_cdf(x):
            returns and stores cdf of a value x
        get_mean():
            returns mean of kumaraswamy distribution and stores it
        get_var():
            returns variance of kumaraswamy distribution and stores it
    """
    def __init__(self,a,b,upper,lower):
        """
        Parameters
        ------
        a,b: Float 
            parameters of Kumaraswamy Distribution
        upper, lower: Float
            Stores the upper and lower limits of the disrtibution
            
        """
        
           
        self.a=a
        self.b=b
        self.upper=upper
        self.lower=lower
        self.scale=upper-lower
            
    def get_z(self,x):
        """
        A function that stores and returnsthe z(normalized)(scaled and shifted) value for corrsponding x
        Parameters
        ------
        x:float
            The value for which we want the pdf(or cdf or z)
        
        
        Returns
        -------
        Z (scaled to (0,1) and shifted)value corresponding to a given x
        
        
        """
        self.z=(x-self.lower)/(self.upper-self.lower)
        return self.z
            
            
  
    
    def get_pdf(self,x):
        """
        A function that stores and returns the pdf  for corrsponding x of given kumaraswamy distribution
        
        Parameters
        ------
        x:float
            The value for which we want the pdf
        
        
        Returns
        -------
        Pdf  for corrsponding x of given kumaraswamy distribution
        
        
        """
        y=self.get_z(x)
        
        d2=kumaraswamy(a=self.a,b=self.b)
        m=d2.pdf(y)
        self.pdf=np.asscalar(m)
        return self.pdf
    
    def get_cdf(self,x):
        """
        A function that stores and returns the cdf  for corrsponding x of given kumaraswamy distribution
        
        Parameters
        ------
        x:float
            The value for which we want the cdf
        
        
        Returns
        -------
        Cdf  for corrsponding x of given kumaraswamy distribution
        
        """
        y=self.get_z(x)
        ## Is this a good practice
       
        d2=kumaraswamy(a=self.a,b=self.b)
        z=d2.cdf(y)
        self.cdf=np.asscalar(z)
        return self.cdf
    
    def get_mean(self):
        """
        A function that returns the mean of the kumaraswamy Distribution
        
        Parameters
        ------
        None
        
        
        Returns
        -------
        Mean of the given Kumaraswmy distribution
        
        """
       
        d2=kumaraswamy(a=self.a,b=self.b)
        l=d2.mean
        self.mean=l*(self.scale)+self.lower
        return self.mean
        
    def get_var(self):
        """
        A function that returns the variance of the kumaraswamy Distribution
        
        Parameters
        ------
        None
        
        
        Returns
        -------
        variance of the given Kumaraswmy distribution
        
        """
        d2=kumaraswamy(self.a,self.b)
        t=d2.var
        self.var=(t)*(self.scale)*(self.scale)
        return self.var
        

In [142]:
obj3=kumaraswamy_scaled(0.2,0.3,5,7)
obj4=kumaraswamy_scaled(0.7,0.2,5,7)

In [143]:
obj3.get_mean()

5.932711602265373